In [1]:
(in-package "ACL2")

 "ACL2"


In [2]:
;; Trust tags needed for quicklisp/HTTP client
(include-book "verified-agent")
(include-book "llm-client"
              :ttags ((:quicklisp) (:quicklisp.osicat) (:quicklisp.dexador) (:http-json) (:llm-client)))
(include-book "mcp-client"
              :ttags ((:quicklisp) (:quicklisp.dexador) (:http-json) (:mcp-client)))

Note (from clause-processors/equality): disabling DISJOIN, DISJOIN2,
CONJOIN and CONJOIN2.


Summary
Form:  ( INCLUDE-BOOK "verified-agent" ...)
Rules: NIL
Time:  0.81 seconds (prove: 0.00, print: 0.00, other: 0.81)
 "/workspaces/verified-agent/src/verified-agent.lisp"

TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

TTAG NOTE (for included book): Adding ttag :QUICKLISP.DEXADOR from book /home/acl2/books/quicklisp/dexador.

TTAG NOTE (for included book): Adding ttag :HTTP-JSON from book /workspaces/verified-agent/src/http-json.

TTAG NOTE (for included book): Adding ttag :LLM-CLIENT from book /workspaces/verified-agent/src/llm-client.

Summary
Form:  ( INCLUDE-BOOK "llm-client" ...)
Rules: NIL
Time:  0.39 seconds (prove: 0.00, print: 0.00, other: 0.39)
 "/workspaces/verified-agent/src/llm-client.lisp"

TTAG NOTE (for included book): Adding ttag :QUICKLISP from book /home/acl2/books/quicklisp/base.

TTAG NOTE (for included book): Adding 

In [3]:
;; Model preferences in order - first match wins
(defconst *model-prefs* '("qwen" "nemotron" "llama" "gemma"))


Summary
Form:  ( DEFCONST *MODEL-PREFS* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MODEL-PREFS*


In [4]:
;; Get full model info and select best loaded completions model
#-skip-interactive
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (prog2$ 
    (if err
        (cw "~%Error getting models: ~s0~%" err)
      (progn$
       (cw "~%Available models: ~x0~%" (len models))
       (cw "Loaded completions models: ~x0~%" 
           (len (filter-loaded-completions-models models)))))
    (mv nil '(value-triple :models-listed) state))))


Available models: 15
Loaded completions models: 1

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
 :MODELS-LISTED


In [5]:
;; Select the best model based on preferences
#-skip-interactive
(make-event
 (mv-let (err models state)
   (llm-list-models-full state)
   (declare (ignorable err))
   (let ((selected (select-completions-model models *model-prefs*)))
     (prog2$
      (if selected
          (cw "~%Selected model: ~s0 (context: ~x1 tokens)~%"
              (model-info->id selected)
              (model-info->loaded-context-length selected))
        (cw "~%WARNING: No suitable model found! Make sure LM Studio is running.~%"))
      (mv nil 
          `(defconst *model-id* 
             ,(if selected (model-info->id selected) "no-model-found"))
          state)))))


Selected model: qwen/qwen3-coder-30b (context: 80000 tokens)

Summary
Form:  ( DEFCONST *MODEL-ID* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)

Summary
Form:  ( MAKE-EVENT (MV-LET ...))
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *MODEL-ID*


In [6]:
(defconst *initial-state*
  (make-agent-state 
    :max-steps 20              ; Allow up to 20 conversation turns
    :token-budget 50000        ; Token budget for tool calls
    :time-budget 3600          ; 1 hour time budget
    :file-access 1             ; Read access to files
    :execute-allowed t         ; Code execution enabled!
    :max-context-tokens 8000   ; Context window size
    :satisfaction 0))          ; Starting satisfaction


Summary
Form:  ( DEFCONST *INITIAL-STATE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *INITIAL-STATE*


In [7]:
(defconst *system-prompt*
  "You are a helpful AI assistant running inside a formally verified ReAct agent framework built in ACL2.

You have access to ACL2 code execution. To run ACL2 code, put it in a fenced code block:

```acl2
(+ 1 2 3)
```

or:

```lisp
(defun factorial (n) (if (zp n) 1 (* n (factorial (1- n)))))
```

I will execute the code and show you the result. You can then continue reasoning or give a final answer.

Be concise. Show your reasoning.")


Summary
Form:  ( DEFCONST *SYSTEM-PROMPT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SYSTEM-PROMPT*


In [8]:
(defconst *agent-v1*
  (init-agent-conversation *system-prompt* *initial-state*))


Summary
Form:  ( DEFCONST *AGENT-V1* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V1*


In [9]:
;; Verify the conversation was initialized
#-skip-interactive
(defconst *check-init*
  (prog2$ (cw "Agent initialized. Messages: ~x0~%" 
              (len (get-messages *agent-v1*)))
          t))

Agent initialized. Messages: 1

Summary
Form:  ( DEFCONST *CHECK-INIT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CHECK-INIT*


In [10]:
(defun show-messages (msgs)
  (declare (xargs :mode :program))
  (if (endp msgs)
      nil
    (let* ((msg (car msgs))
           (role (chat-message->role msg))
           (content (chat-message->content msg))
           (role-str (chat-role-case role
                       :system "SYSTEM"
                       :user "USER"
                       :assistant "ASSISTANT"
                       :tool "TOOL")))
      (prog2$ (cw "~%[~s0]~%~s1~%" role-str content)
              (show-messages (cdr msgs))))))


Summary
Form:  ( DEFUN SHOW-MESSAGES ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 SHOW-MESSAGES


In [11]:
(defun show-conversation (st)
  (declare (xargs :mode :program))
  (prog2$ (cw "~%========== Conversation ==========")
          (prog2$ (show-messages (get-messages st))
                  (cw "~%===================================~%"))))


Summary
Form:  ( DEFUN SHOW-CONVERSATION ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 SHOW-CONVERSATION


In [12]:
#-skip-interactive
(defconst *agent-v2*
  (add-user-msg "Hello! Can you explain what a formally verified agent is?" 
                *agent-v1*))


Summary
Form:  ( DEFCONST *AGENT-V2* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V2*


In [13]:
;; Show the conversation so far
#-skip-interactive
(defconst *show-v2*
  (show-conversation *agent-v2*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.  You have access to ACL2 code execution.
To run ACL2 code, put it in a fenced code block:  ```acl2 (+ 1 2 3)
```  or:  ```lisp (defun factorial (n) (if (zp n) 1 (* n (factorial
(1- n))))) ```  I will execute the code and show you the result. You
can then continue reasoning or give a final answer.  Be concise. Show
your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?


Summary
Form:  ( DEFCONST *SHOW-V2* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-V2*


In [14]:
;; This calls the LLM and adds the response to conversation
;; Requires LM Studio running!
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *agent-v2*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *agent-v3* *agent-v2*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *agent-v3* 
                        (add-assistant-msg ,response *agent-v2*)) 
                 state)))))


Assistant: A formally verified agent is an AI system whose behavior
and decision-making processes have been mathematically proven to satisfy
specific properties or requirements. This means that instead of relying
solely on testing or empirical validation, the agent's correctness
is established through rigorous mathematical proofs.

Key characteristics of formally verified agents:

1. **Mathematical Proofs**: The agent's algorithms and decision-making
logic are backed by formal proofs that demonstrate they meet specified
criteria

2. **Guaranteed Properties**: We can mathematically guarantee that
certain desirable properties hold, such as:
   - Safety constraints (never taking harmful actions)
   - Liveness properties (eventually achieving goals)
   - Correctness of computations

3. **ACL2 Framework**: As mentioned in your query, ACL2 (A Computational
Logic for Applicative Common Lisp) is a formal verification system
that can be used to specify and prove properties of computational sys

In [15]:
;; Show updated conversation
#-skip-interactive
(defconst *show-v3*
  (show-conversation *agent-v3*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.  You have access to ACL2 code execution.
To run ACL2 code, put it in a fenced code block:  ```acl2 (+ 1 2 3)
```  or:  ```lisp (defun factorial (n) (if (zp n) 1 (* n (factorial
(1- n))))) ```  I will execute the code and show you the result. You
can then continue reasoning or give a final answer.  Be concise. Show
your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or requirements. This means that instead of relying solely
on testing or empirical validation, the agent's correctness is established
through rigorous mathematical proofs.

Key characteristics of formally verified agents:

1. **Mathematical Proofs**: The agent's algorithms and decision-mak

In [16]:
#-skip-interactive
(defconst *agent-v4*
  (add-user-msg "What properties have been proven about this agent?" 
                *agent-v3*))


Summary
Form:  ( DEFCONST *AGENT-V4* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V4*


In [17]:
#-skip-interactive
(defconst *show-v4*
  (show-conversation *agent-v4*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.  You have access to ACL2 code execution.
To run ACL2 code, put it in a fenced code block:  ```acl2 (+ 1 2 3)
```  or:  ```lisp (defun factorial (n) (if (zp n) 1 (* n (factorial
(1- n))))) ```  I will execute the code and show you the result. You
can then continue reasoning or give a final answer.  Be concise. Show
your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or requirements. This means that instead of relying solely
on testing or empirical validation, the agent's correctness is established
through rigorous mathematical proofs.

Key characteristics of formally verified agents:

1. **Mathematical Proofs**: The agent's algorithms and decision-mak

In [18]:
;; Call LLM and add response to conversation
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *agent-v4*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *agent-v5* *agent-v4*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *agent-v5* 
                        (add-assistant-msg ,response *agent-v4*)) 
                 state)))))


Assistant: Looking at the ACL2 framework mentioned in the query, I
can explain what properties would typically be proven about a formally
verified agent in this context.

In an ACL2-based formally verified agent, the following types of properties
might be proven:

1. **Safety Properties**: 
   - The agent will never violate specified constraints
   - Agent actions remain within safe operating boundaries

2. **Correctness Properties**:
   - The agent's reasoning processes follow valid logical inference
   - Computational results are mathematically correct

3. **Behavioral Properties**:
   - Agent responses are consistent with its specification
   - The agent maintains appropriate state transitions

4. **Reactivity Properties**:
   - The agent properly responds to inputs within specified time bounds
   - Agent actions are triggered correctly based on environmental conditions

However, since this is a theoretical ACL2 verification framework and
not a specific implemented agent, I cannot 

In [19]:
#-skip-interactive
(defconst *show-v5*
  (show-conversation *agent-v5*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.  You have access to ACL2 code execution.
To run ACL2 code, put it in a fenced code block:  ```acl2 (+ 1 2 3)
```  or:  ```lisp (defun factorial (n) (if (zp n) 1 (* n (factorial
(1- n))))) ```  I will execute the code and show you the result. You
can then continue reasoning or give a final answer.  Be concise. Show
your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or requirements. This means that instead of relying solely
on testing or empirical validation, the agent's correctness is established
through rigorous mathematical proofs.

Key characteristics of formally verified agents:

1. **Mathematical Proofs**: The agent's algorithms and decision-mak

In [20]:
#-skip-interactive
(defconst *agent-v6*
  (add-user-msg "That's impressive! How does the context management work?" 
                *agent-v5*))


Summary
Form:  ( DEFCONST *AGENT-V6* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-V6*


In [21]:
;; Call LLM for Turn 3
#-skip-interactive
(make-event
 (mv-let (err response state)
   (llm-chat-completion *model-id* (get-messages *agent-v6*) state)
   (if err
       (prog2$ (cw "~%LLM Error: ~s0~%" err)
               (mv nil `(defconst *agent-v7* *agent-v6*) state))
     (prog2$ (cw "~%Assistant: ~s0~%" response)
             (mv nil `(defconst *agent-v7* 
                        (add-assistant-msg ,response *agent-v6*)) 
                 state)))))


Assistant: In a formally verified ReAct agent framework using ACL2,
context management would involve several key components that are mathematicall\
y specified and proven:

1. **State Representation**: The agent's context would be formally
defined as a mathematical structure (likely a tuple or record type)
containing:
   - Current observation/state
   - Memory of previous interactions
   - Agent's internal beliefs/representations
   - Task-specific context information

2. **Context Updates**: The mechanism for updating context would be
formally specified with:
   - Well-defined transition functions
   - Preservation of important properties during updates
   - Consistency guarantees between different context components

3. **ReAct Loop Integration**: The context management would be proven
to:
   - Properly interface with the reasoning (R) and action (A) components
   - Maintain consistency between observations, thoughts, and actions
   - Support the iterative nature of ReAct planning



In [22]:
#-skip-interactive
(defconst *show-v7*
  (show-conversation *agent-v7*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.  You have access to ACL2 code execution.
To run ACL2 code, put it in a fenced code block:  ```acl2 (+ 1 2 3)
```  or:  ```lisp (defun factorial (n) (if (zp n) 1 (* n (factorial
(1- n))))) ```  I will execute the code and show you the result. You
can then continue reasoning or give a final answer.  Be concise. Show
your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or requirements. This means that instead of relying solely
on testing or empirical validation, the agent's correctness is established
through rigorous mathematical proofs.

Key characteristics of formally verified agents:

1. **Mathematical Proofs**: The agent's algorithms and decision-mak

In [23]:
#-skip-interactive
(defconst *context-check*
  (let* ((msgs (get-messages *agent-v7*))
         (char-len (messages-char-length msgs))
         (token-est (messages-token-estimate msgs))
         (max-tokens (agent-state->max-context-tokens *agent-v7*)))
    (prog2$ (cw "~%Context Usage:~%")
      (prog2$ (cw "  Total characters: ~x0~%" char-len)
        (prog2$ (cw "  Estimated tokens: ~x0~%" token-est)
          (prog2$ (cw "  Max tokens: ~x0~%" max-tokens)
            (prog2$ (cw "  Safety margin: ~x0~%" *context-safety-margin*)
              (cw "  Fits in context: ~x0~%" 
                  (messages-fit-p msgs max-tokens)))))))))


Context Usage:
  Total characters: 4911
  Estimated tokens: 1228
  Max tokens: 8000
  Safety margin: 500
  Fits in context: T

Summary
Form:  ( DEFCONST *CONTEXT-CHECK* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CONTEXT-CHECK*


In [24]:
;; Simulate calling a file-read tool
#-skip-interactive
(defconst *agent-with-tool*
  (add-tool-result 
    "File contents of verified-agent.lisp (first 500 chars):
; Verified ReAct Agent - ACL2 Implementation
; This file implements a formally verified ReAct agent...
(in-package \"ACL2\")
(include-book \"std/util/define\" :dir :system)
..." 
    *agent-v7*))


Summary
Form:  ( DEFCONST *AGENT-WITH-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-WITH-TOOL*


In [25]:
#-skip-interactive
(defconst *show-tool*
  (show-conversation *agent-with-tool*))


========== Conversation ==========
[SYSTEM]
You are a helpful AI assistant running inside a formally verified ReAct
agent framework built in ACL2.  You have access to ACL2 code execution.
To run ACL2 code, put it in a fenced code block:  ```acl2 (+ 1 2 3)
```  or:  ```lisp (defun factorial (n) (if (zp n) 1 (* n (factorial
(1- n))))) ```  I will execute the code and show you the result. You
can then continue reasoning or give a final answer.  Be concise. Show
your reasoning.

[USER]
Hello! Can you explain what a formally verified agent is?

[ASSISTANT]
A formally verified agent is an AI system whose behavior and decision-
making processes have been mathematically proven to satisfy specific
properties or requirements. This means that instead of relying solely
on testing or empirical validation, the agent's correctness is established
through rigorous mathematical proofs.

Key characteristics of formally verified agents:

1. **Mathematical Proofs**: The agent's algorithms and decision-mak

In [26]:
;; Strip leading whitespace from character list
#-skip-interactive
(defun strip-leading-ws (lst)
  (declare (xargs :mode :program))
  (if (endp lst) nil
    (if (member (car lst) '(#\Space #\Tab #\Newline #\Return))
        (strip-leading-ws (cdr lst))
      lst)))


Summary
Form:  ( DEFUN STRIP-LEADING-WS ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 STRIP-LEADING-WS


In [27]:
;; Trim whitespace from string
#-skip-interactive
(defun my-string-trim (str)
  (declare (xargs :mode :program))
  (let* ((chars (coerce str 'list))
         (trimmed (strip-leading-ws chars))
         (rev-trimmed (strip-leading-ws (reverse trimmed))))
    (coerce (reverse rev-trimmed) 'string)))


Summary
Form:  ( DEFUN MY-STRING-TRIM ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 MY-STRING-TRIM


In [28]:
;; Extract first ```acl2 or ```lisp code block from text
#-skip-interactive
(defun extract-code-block (response)
  (declare (xargs :mode :program))
  (let* ((acl2-start (search "```acl2" response))
         (lisp-start (search "```lisp" response))
         (start-marker (cond ((and acl2-start lisp-start)
                              (if (< acl2-start lisp-start) "```acl2" "```lisp"))
                             (acl2-start "```acl2")
                             (lisp-start "```lisp")
                             (t nil)))
         (start-pos (cond ((and acl2-start lisp-start)
                           (min acl2-start lisp-start))
                          (acl2-start acl2-start)
                          (lisp-start lisp-start)
                          (t nil))))
    (if (not start-pos)
        (mv nil "")
      (let* ((content-start (+ start-pos (length start-marker)))
             (newline-pos (search (coerce '(#\Newline) 'string)
                                  (subseq response content-start (length response))))
             (code-start (if newline-pos (+ content-start newline-pos 1) content-start))
             (rest (subseq response code-start (length response)))
             (end-pos (search "```" rest)))
        (if (not end-pos)
            (mv nil "")
          (mv t (my-string-trim (subseq rest 0 end-pos))))))))


Summary
Form:  ( DEFUN EXTRACT-CODE-BLOCK ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 EXTRACT-CODE-BLOCK


In [29]:
;; Execute ACL2 code via MCP
;; Uses mcp-acl2-execute which handles multiple forms and checks syntax first
#-skip-interactive
(defun execute-acl2-code (code mcp-conn state)
  (declare (xargs :mode :program :stobjs state))
  (if (not (mcp-connection-p mcp-conn))
      (mv "No MCP connection" "" state)
    (mcp-acl2-execute mcp-conn code state)))


Summary
Form:  ( DEFUN EXECUTE-ACL2-CODE ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 EXECUTE-ACL2-CODE


In [30]:
;; ReAct step: call LLM, check for code, execute if found, loop until done
#-skip-interactive
(defun react-loop (agent-st model-id mcp-conn max-steps state)
  "Execute ReAct loop: LLM -> extract code -> execute -> repeat until no code block"
  (declare (xargs :mode :program :stobjs state))
  (if (zp max-steps)
      (prog2$ (cw "~%[Max steps reached]~%")
              (mv agent-st state))
    (mv-let (err response state)
      (llm-chat-completion model-id (get-messages agent-st) state)
      (if err
          (prog2$ (cw "~%LLM Error: ~s0~%" err)
                  (mv agent-st state))
        (let ((agent-st (add-assistant-msg response agent-st)))
          (prog2$ (cw "~%Assistant: ~s0~%" response)
            (mv-let (found? code)
              (extract-code-block response)
              (if (not found?)
                  ;; No code block - done with this turn
                  (mv agent-st state)
                ;; Execute code and continue
                (prog2$ (cw "~%[Executing: ~s0]~%" code)
                  (mv-let (exec-err result state)
                    (execute-acl2-code code mcp-conn state)
                    (let* ((tool-result (if exec-err
                                            (concatenate 'string "Error: " exec-err)
                                          result))
                           ;; Truncate long results for display
                           (display-result (if (> (length tool-result) 300)
                                               (concatenate 'string 
                                                 (subseq tool-result 0 300) "...")
                                             tool-result))
                           (agent-st (add-tool-result tool-result agent-st)))
                      (prog2$ (cw "~%Result: ~s0~%" display-result)
                        ;; Continue ReAct loop
                        (react-loop agent-st model-id mcp-conn (1- max-steps) state)))))))))))))


Summary
Form:  ( DEFUN REACT-LOOP ...)
Rules: NIL
Time:  0.02 seconds (prove: 0.00, print: 0.00, other: 0.02)
 REACT-LOOP


In [31]:
#-skip-interactive
(defun chat-turn (user-msg agent-st model-id mcp-conn state)
  "Execute one chat turn with ReAct: add user message, run ReAct loop"
  (declare (xargs :mode :program :stobjs state))
  (let ((st-with-user (add-user-msg user-msg agent-st)))
    (react-loop st-with-user model-id mcp-conn 5 state)))


Summary
Form:  ( DEFUN CHAT-TURN ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 CHAT-TURN


In [32]:
;; Create a tool spec for demonstration
#-skip-interactive
(defconst *read-file-tool*
  (make-tool-spec
    :name 'read-file
    :required-access 1      ; Needs read access
    :requires-execute nil
    :token-cost 100
    :time-cost 5))


Summary
Form:  ( DEFCONST *READ-FILE-TOOL* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *READ-FILE-TOOL*


In [33]:
;; Check if we can invoke this tool
#-skip-interactive
(defconst *can-read-file*
  (prog2$ (cw "~%Can invoke read-file tool: ~x0~%" 
              (can-invoke-tool-p *read-file-tool* *agent-v7*))
          (can-invoke-tool-p *read-file-tool* *agent-v7*)))


Can invoke read-file tool: T

Summary
Form:  ( DEFCONST *CAN-READ-FILE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *CAN-READ-FILE*


In [34]:
;; Simulate a ReAct step with conversation
#-skip-interactive
(defconst *agent-after-react*
  (if (and (not (must-respond-p *agent-v7*))
           (can-invoke-tool-p *read-file-tool* *agent-v7*))
      (react-step-with-conversation
        (agent-action-tool-call 'read-file "verified-agent.lisp")
        *read-file-tool*
        "I'll read the verified-agent.lisp file to show you the implementation."
        "File read successfully. Contents: (in-package ACL2)..."
        *agent-v7*)
    *agent-v7*))


Summary
Form:  ( DEFCONST *AGENT-AFTER-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *AGENT-AFTER-REACT*


In [35]:
#-skip-interactive
(defconst *show-react*
  (prog2$ (cw "~%After ReAct step:~%")
    (prog2$ (cw "  Step counter: ~x0~%" 
                (agent-state->step-counter *agent-after-react*))
      (prog2$ (cw "  Token budget: ~x0~%" 
                  (agent-state->token-budget *agent-after-react*))
        (cw "  Message count: ~x0~%" 
            (len (get-messages *agent-after-react*)))))))


After ReAct step:
  Step counter: 1
  Token budget: 49900
  Message count: 9

Summary
Form:  ( DEFCONST *SHOW-REACT* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *SHOW-REACT*


In [36]:
#-skip-interactive
(defconst *demo-complete*
  (prog2$ (cw "~%~%========================================~%")
    (prog2$ (cw "Chat Demo Complete!~%")
      (prog2$ (cw "========================================~%")
        (prog2$ (cw "~%Key points demonstrated:~%")
          (prog2$ (cw "  1. Agent state initialization~%")
            (prog2$ (cw "  2. Conversation management (add-user-msg, add-assistant-msg)~%")
              (prog2$ (cw "  3. Context length tracking~%")
                (prog2$ (cw "  4. Tool result handling~%")
                  (prog2$ (cw "  5. ReAct step with conversation~%")
                    (prog2$ (cw "  6. State preservation verification~%")
                      (cw "~%For live LLM chat, ensure LM Studio is running at:~%  http://host.docker.internal:1234~%"))))))))))))



Chat Demo Complete!

Key points demonstrated:
  1. Agent state initialization
  2. Conversation management (add-user-msg, add-assistant-msg)
  3. Context length tracking
  4. Tool result handling
  5. ReAct step with conversation
  6. State preservation verification

For live LLM chat, ensure LM Studio is running at:
  http://host.docker.internal:1234

Summary
Form:  ( DEFCONST *DEMO-COMPLETE* ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 *DEMO-COMPLETE*


In [37]:
;; Read a line from standard input character by character
#-skip-interactive
(defun read-line-chars (acc state)
  "Accumulate characters until newline or EOF."
  (declare (xargs :mode :program :stobjs state))
  (mv-let (char state)
    (read-char$ *standard-ci* state)
    (cond ((null char)  ; EOF
           (mv (if (null acc) nil (coerce (reverse acc) 'string)) state))
          ((eql char #\Newline)
           (mv (coerce (reverse acc) 'string) state))
          (t (read-line-chars (cons char acc) state)))))


Summary
Form:  ( DEFUN READ-LINE-CHARS ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 READ-LINE-CHARS


In [38]:
#-skip-interactive
(defun read-line-from-user (state)
  "Read a line from standard input, return (mv line state)."
  (declare (xargs :mode :program :stobjs state))
  (read-line-chars nil state))


Summary
Form:  ( DEFUN READ-LINE-FROM-USER ...)
Rules: NIL
Time:  0.00 seconds (prove: 0.00, print: 0.00, other: 0.00)
 READ-LINE-FROM-USER


In [39]:
;; Helper must be defined before main function
#-skip-interactive
(defun interactive-chat-loop-aux (agent-st model-id mcp-conn state)
  "Helper for interactive chat loop with code execution."
  (declare (xargs :mode :program :stobjs state))
  (prog2$ (cw "~%You: ")  ; prompt
    (mv-let (input state)
      (read-line-from-user state)
      (if (or (null input)
              (equal input "/exit")
              (equal input "/quit"))
          (prog2$ (cw "~%Goodbye!~%")
                  (mv agent-st state))
        (mv-let (new-agent state)
          (chat-turn input agent-st model-id mcp-conn state)
          (interactive-chat-loop-aux new-agent model-id mcp-conn state))))))


Summary
Form:  ( DEFUN INTERACTIVE-CHAT-LOOP-AUX ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 INTERACTIVE-CHAT-LOOP-AUX


In [40]:
;; Interactive chat loop - runs until user types /exit
#-skip-interactive
(defun interactive-chat-loop (agent-st model-id state)
  "Run an interactive ReAct chat loop with code execution. Type /exit to quit."
  (declare (xargs :mode :program :stobjs state))
  (prog2$ (cw "~%========================================~%")
    (prog2$ (cw "Interactive ReAct Chat with Code Execution~%")
      (prog2$ (cw "(type /exit to quit)~%")
        (prog2$ (cw "========================================~%")
          (prog2$ (cw "~%Connecting to MCP server...~%")
            (mv-let (mcp-err mcp-conn state)
              (mcp-connect *mcp-default-endpoint* state)
              (if mcp-err
                  (prog2$ (cw "MCP connection failed: ~s0~%" mcp-err)
                    (prog2$ (cw "Code execution disabled.~%")
                      (interactive-chat-loop-aux agent-st model-id nil state)))
                (prog2$ (cw "MCP connected.~%")
                  (prog2$ (if (mcp-connection-has-acl2-session-p mcp-conn)
                              (cw "ACL2 session: ~s0~%" 
                                  (mcp-connection->acl2-session-id mcp-conn))
                            (cw "Warning: No persistent ACL2 session (slower mode).~%"))
                    (interactive-chat-loop-aux agent-st model-id mcp-conn state)))))))))))


Summary
Form:  ( DEFUN INTERACTIVE-CHAT-LOOP ...)
Rules: NIL
Time:  0.01 seconds (prove: 0.00, print: 0.00, other: 0.01)
 INTERACTIVE-CHAT-LOOP
